## Import dependencies

In [ ]:
from tensorflow.keras.utils import get_file
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

### Define constants

In [ ]:
DROPOUT = 0.2
NUM_HEADS = 4
NUM_LAYER = 4
BLOCK_SIZE = 64
NUM_EMBED = 128
BATCH_SIZE = 64
EVAL_ITERS = 200
EVAL_INTERVAL = 500
LEARNING_RATE = 1e-3
MAX_ITERATION = 10000
MAX_NEW_TOKENS = 1000

### Get the text file

In [ ]:
path_to_file = get_file("shakespeare.txt",
                        "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")

text = open(path_to_file, "r").read()

1115394/1115394 [==============================] - 0s 0us/step


In [ ]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
print(f"Length of dataset in characters: {len(text)}")

Length of dataset in characters: 1115394


### Build the vocabulary

In [ ]:
characters = sorted(list(set(text)))
VOCAB_SIZE = len(characters)
print(f"""VOCABULARY: {"".join(characters)}
VOCABULARY SIZE: {VOCAB_SIZE}""")

VOCABULARY: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
VOCABULARY SIZE: 65


In [ ]:
string_to_int = {ch:i for i,ch in enumerate(characters)}
int_to_string = {i:ch for i,ch in enumerate(characters)}

### Label encoder  
#### text-to-numeric vector conversion

In [ ]:
encode = lambda string: [string_to_int[char] for char in string]
decode = lambda integer: "".join([int_to_string[num] for num in integer])

In [ ]:
string = "Hello I am Soumodeep"
integer = encode(string)

print(integer,"\n",decode(integer))

[20, 43, 50, 50, 53, 1, 21, 1, 39, 51, 1, 31, 53, 59, 51, 53, 42, 43, 43, 54] 
 Hello I am Soumodeep


### Store data into tensor

In [ ]:
data= torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


### Training and Validation split

In [ ]:
n = int(0.9*data.shape[0])
training_data = data[:n]
validation_data = data[n:]

In [ ]:
x = training_data[:BLOCK_SIZE+1]

for i in range(BLOCK_SIZE):
    context = x[:i+1]
    target = x[i+1]
    print(f"Input {context} => Output {target}")

Input tensor([18]) => Output 47
Input tensor([18, 47]) => Output 56
Input tensor([18, 47, 56]) => Output 57
Input tensor([18, 47, 56, 57]) => Output 58
Input tensor([18, 47, 56, 57, 58]) => Output 1
Input tensor([18, 47, 56, 57, 58,  1]) => Output 15
Input tensor([18, 47, 56, 57, 58,  1, 15]) => Output 47
Input tensor([18, 47, 56, 57, 58,  1, 15, 47]) => Output 58
Input tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]) => Output 47
Input tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47]) => Output 64
Input tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64]) => Output 43
Input tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43]) => Output 52
Input tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52]) => Output 10
Input tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10]) => Output 0
Input tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0]) => Output 14
Input tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14]) => Output 43
Input tens

In [ ]:
def get_batch(data):
    """
    This function takes a corpus and returns a batch of input and 
    output strings where outputs are one block shifted to the right.
    """
    
    idx = torch.randint(len(data)-BLOCK_SIZE, (BATCH_SIZE,))
    
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in idx])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in idx])
    
    return x,y

In [ ]:
x, y = get_batch(training_data)
print(f"""Input {x}
Input shape {x.shape}
Output {y}
Output shape {y.shape}""")

Input tensor([[50, 43,  8,  ..., 53, 51, 43],
        [ 1, 58, 46,  ..., 33, 25, 14],
        [42,  1, 61,  ..., 51,  1, 52],
        ...,
        [58, 56, 43,  ..., 13, 52, 53],
        [44, 50, 47,  ..., 56, 43, 39],
        [43,  8,  0,  ...,  0, 25, 63]])
Input shape torch.Size([64, 64])
Output tensor([[43,  8,  1,  ..., 51, 43, 52],
        [58, 46, 43,  ..., 25, 14, 17],
        [ 1, 61, 47,  ...,  1, 52, 53],
        ...,
        [56, 43, 61,  ..., 52, 53, 52],
        [50, 47, 43,  ..., 43, 39, 58],
        [ 8,  0,  0,  ..., 25, 63,  1]])
Output shape torch.Size([64, 64])


In [ ]:
for b in range(BATCH_SIZE):
    for t in range(BLOCK_SIZE):
        context = x[b,:t+1]
        target = y[b,t]
        
        print(f"Input {context} -> Target {target}")

Streaming output truncated to the last 5000 lines.
Input tensor([51, 54, 43, 39, 41, 46,  5, 42,  1, 39, 52, 42,  1, 40, 39, 44, 44, 50,
        43, 42,  1, 46, 43, 56, 43,  6]) -> Target 0
Input tensor([51, 54, 43, 39, 41, 46,  5, 42,  1, 39, 52, 42,  1, 40, 39, 44, 44, 50,
        43, 42,  1, 46, 43, 56, 43,  6,  0]) -> Target 28
Input tensor([51, 54, 43, 39, 41, 46,  5, 42,  1, 39, 52, 42,  1, 40, 39, 44, 44, 50,
        43, 42,  1, 46, 43, 56, 43,  6,  0, 28]) -> Target 47
Input tensor([51, 54, 43, 39, 41, 46,  5, 42,  1, 39, 52, 42,  1, 40, 39, 44, 44, 50,
        43, 42,  1, 46, 43, 56, 43,  6,  0, 28, 47]) -> Target 43
Input tensor([51, 54, 43, 39, 41, 46,  5, 42,  1, 39, 52, 42,  1, 40, 39, 44, 44, 50,
        43, 42,  1, 46, 43, 56, 43,  6,  0, 28, 47, 43]) -> Target 56
Input tensor([51, 54, 43, 39, 41, 46,  5, 42,  1, 39, 52, 42,  1, 40, 39, 44, 44, 50,
        43, 42,  1, 46, 43, 56, 43,  6,  0, 28, 47, 43, 56]) -> Target 41
Input tensor([51, 54, 43, 39, 41, 46,  5, 42,  1, 

### Evaluation method
#### During training, after certain interval it evaluates the performance of the model

In [ ]:
@torch.no_grad()
def estimate_loss():
    outcome = {}
    model.eval()
    for split in ["train","validation"]:
        losses = torch.zeros(EVAL_ITERS)
        for k in range(EVAL_ITERS):
            if split == "train":
                x,y = get_batch(training_data)
            else: 
                x,y = get_batch(validation_data)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        outcome[split] = losses.mean()
    model.train()
    return outcome

In [ ]:
class head(nn.Module):
    """
    This class creates a head of a self-attention model
    """
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(NUM_EMBED, head_size, bias=False)
        self.query = nn.Linear(NUM_EMBED, head_size, bias=False)
        self.value = nn.Linear(NUM_EMBED, head_size, bias=False)
        self.register_buffer("tril",torch.tril(torch.ones(BLOCK_SIZE, 
                                                          BLOCK_SIZE)))
        self.dropout = nn.Dropout(DROPOUT)
        
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)                    # Dimension: (B,T,C)
        q = self.query(x)                  # Dimension: (B,T,C)
        
        # computating attention scores
        # Attention(k,q,v) = softmax((q @ k.T) / sqrt(c)) @ v
        weights = q @ k.transpose(-2,-1) * C**-0.5
        weights = weights.masked_fill(self.tril[:T,:T] == 0, 
                                      float('-inf'))
        weights = F.softmax(weights, dim=-1)
        
        weights = self.dropout(weights)
        
        v = self.value(x)
        outcome = weights @ v
        
        return outcome

In [ ]:
class multiple_head_attention(nn.Module):
    """
    This class instance can create multiple attention head 
    instances and run them parallely.
    """
    
    def __init__(self, head_size):
        super().__init__()
        self.heads = nn.ModuleList([head(head_size) for _ in range(NUM_HEADS)])
        self.projection = nn.Linear(NUM_EMBED, NUM_EMBED)
        self.dropout = nn.Dropout(DROPOUT)
        
    def forward(self, x):
        outcome = torch.cat([h(x) for h in self.heads], dim=-1)
        outcome = self.projection(outcome)
        
        return outcome

In [ ]:
class forward_propagation(nn.Module):
    """
    A Multi-layered Perceptron
    """
    
    def __init__(self):
        super().__init__()
        self.mlp = nn.Sequential(nn.Linear(NUM_EMBED, NUM_EMBED*4), 
                                 nn.ReLU(),
                                 nn.Linear(NUM_EMBED*4, NUM_EMBED),
                                 nn.Dropout(DROPOUT))
        
    def forward(self, x):
        return self.mlp(x)

In [ ]:
class block(nn.Module):
    """
    This is a transformer block where the communication i.e., 
    Attention scores are obtained and then passed into a simple
    feedforward perceptron model.
    """
    
    def __init__(self):
        super().__init__()
        head_size = NUM_EMBED // NUM_HEADS
        self.self_attention_heads = multiple_head_attention(head_size)
        self.feedforward = forward_propagation()
        self.layernorm1 = nn.LayerNorm(NUM_EMBED)
        self.layernorm2 = nn.LayerNorm(NUM_EMBED)
        
        
    def forward(self, x):
        x = x + self.self_attention_heads(self.layernorm1(x))
        x = x + self.feedforward(self.layernorm2(x))
        
        return x

In [ ]:
class bigram_language_model(nn.Module):
    """
    This model is a bigram language model. It takes a set of tokens as input.
    Creates an embedding space. Generates new tokens based on given token.
    """
    
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(VOCAB_SIZE, NUM_EMBED)
        self.position_embedding_table = nn.Embedding(BLOCK_SIZE, NUM_EMBED)
        self.attention_blocks = nn.Sequential(*[block() for _ in range(NUM_LAYER)])
        self.layernorm = nn.LayerNorm(NUM_EMBED)
        self.linear_head = nn.Linear(NUM_EMBED, VOCAB_SIZE)

        
    def forward(self, idx, targets=None):
        B,T = idx.shape
        
        token_embeddings = self.token_embedding_table(idx)                    # Dimesion : (B,T,C)
        position_embeddings = self.position_embedding_table(torch.arange(T))  # Dimesion : (T,C) 
        x = token_embeddings + position_embeddings 
        x = self.attention_blocks(x)
        x = self.layernorm(x)
        logits = self.linear_head(x)                                          
        
        if targets is None: loss = None
        else:
            
            # B: batch_size, T: sequence length, C: embedding vector dimension
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is the (B,T) array of indices in the current context

        for _ in range(max_new_tokens):
            
            # slice the idx upto last block_size length
            idx_conditioned = idx[:, -BLOCK_SIZE:]
            
            # getting the predictions
            logits, loss = self(idx_conditioned)
            
            # focus on the last time step 
            logits = logits[:,-1,:]
            
            # apply softmax to get the probabilities
            probabilities = F.softmax(logits, dim=-1)
            
            # sample from the distribution
            idx_next = torch.multinomial(probabilities, num_samples=1)
            
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)
        
        return idx

In [ ]:
model = bigram_language_model()
logits, loss = model(x,y)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

for steps in range(MAX_ITERATION):
    
    if steps % EVAL_INTERVAL == 0:
        losses = estimate_loss()
        print(f"Step {steps+1} -- train_loss {losses['train']:.4f} -- validation_loss {losses['validation']:.4f}")
    
    # generates batch of training data
    x, y = get_batch(training_data)
    
    # forward pass 
    #evaluate the loss
    logits, loss = model(x,y)
    
    # resets gradients of optimizer
    optimizer.zero_grad(set_to_none=True)
    
    # computes gradients for backprop
    loss.backward()
    
    #updates parameters by backprop
    optimizer.step()
    
print(loss.item())

Step 1 -- train_loss 4.3355 -- validation_loss 4.3434
Step 501 -- train_loss 1.8924 -- validation_loss 1.9948
Step 1001 -- train_loss 1.6461 -- validation_loss 1.8140
Step 1501 -- train_loss 1.5453 -- validation_loss 1.7098
Step 2001 -- train_loss 1.4838 -- validation_loss 1.6692
Step 2501 -- train_loss 1.4480 -- validation_loss 1.6407
Step 3001 -- train_loss 1.4233 -- validation_loss 1.6208
Step 3501 -- train_loss 1.3999 -- validation_loss 1.5969
Step 4001 -- train_loss 1.3879 -- validation_loss 1.5951
Step 4501 -- train_loss 1.3665 -- validation_loss 1.5731
Step 5001 -- train_loss 1.3557 -- validation_loss 1.5682
Step 5501 -- train_loss 1.3469 -- validation_loss 1.5573
Step 6001 -- train_loss 1.3370 -- validation_loss 1.5530
Step 6501 -- train_loss 1.3310 -- validation_loss 1.5460
Step 7001 -- train_loss 1.3181 -- validation_loss 1.5399
Step 7501 -- train_loss 1.3147 -- validation_loss 1.5383
Step 8001 -- train_loss 1.3065 -- validation_loss 1.5426
Step 8501 -- train_loss 1.3046 -- v

In [ ]:
def generate_from_token(model):
    idx = torch.zeros((1, 1), dtype=torch.long)
    gen_tokens = model.generate(idx, 
                                max_new_tokens=MAX_NEW_TOKENS)[0].tolist()
    return decode(gen_tokens)

print(generate_from_token(model))


Thundo-bellike full of corrupt bittering fantard.
For noisome and spirit! and well! Ist thou did,
What raised tombles myself.

Lord Marsare Hastily lour, thour such a broats upon
the shemorse hath wadom, as I give to my inform,
Exceive Hereford, i' the tide beyong from you,
For this combles too, that your carrions,
And what the doils his never crown;
And far die, yet pensuage, and oft the Captain Delight
As it off here that and day the Tower,
Which a Paulina to consul, or to Juliet?

Sirs, Murderer:
Dost you shall not so.

POMPER?

JULIET:
No; he should me now? who: I smay, I know? will not such
privator? what of their taken yess he is: yet
His must this off this him I had prove
a cheepinion shephy kinders puddink, let us no; think,
His likelf me can prend the horse speeds egarled:
How speaks' sak your young so, for I shall
The lamentain!

PARIS:
My lord, for weigness is for thee to keep now,
Mow he is days, with his ears.' let's fice
His mistress, even vain. Isabel so, resolution.

P